## 4.9 V/P戦略の実装

In [1]:
import pandas as pd
import statsmodels.api as sm

### コード 4.1 ライブラリの読み込みと入力データの準備

In [2]:
# 各銘柄のβ推定などに利用するffMonthly.csvを読み込み
ffMonthly = pd.read_csv('./data/ffMonthly.csv', parse_dates=['month'])
ffMonthly['month'] = ffMonthly['month'].dt.to_period('M')
print(ffMonthly)
##        month   RMRF   SMB   HML    RF
## 0    1990-07  20.67 -1.56 -5.16  0.68
## 1    1990-08 -13.69 -3.63  0.98  0.66

# 必要なデータの選択(行と列の選択)
# 産業A(tickerがAから始まる) の月次リターンデータを読み込み
stockMonthly = pd.read_csv('./data/stockMonthly.csv', parse_dates=['month'])
stockMonthly['month'] = stockMonthly['month'].dt.to_period('M')
stockMonthly = stockMonthly[stockMonthly['industry'] == 'A']

# β推定のために，2008/7-2013/6の60ヶ月のデータを選択
df = stockMonthly[(stockMonthly['month'] >= '2008-7') &
                  (stockMonthly['month'] <= '2013-6')].copy()
df = df[['ticker', 'month', 'close', 'return', 'share']]
# ffMonthlyを結合し各銘柄の月次超過リターン (RIRF)を求める
df = pd.merge(df, ffMonthly[['month', 'RMRF', 'RF']], on='month')
df['RIRF'] = df['return'] - df['RF']
print(df)
##      ticker    month  close     return      share   RMRF    RF       RIRF
## 0     A0001  2008-07   2820  -7.571288   15815814   3.24  0.15  -7.721288
## 1     A0002  2008-07  12484  16.270840    5046567   3.24  0.15  16.120840

# V/P推定後の運用期間データの選択
futureRet = stockMonthly[(stockMonthly['month'] >= '2013-7') &
                         (stockMonthly['month'] <= '2014-6')]
futureRet = futureRet[['ticker', 'month', 'return']]
print(futureRet)
##       ticker    month     return
## 270    A0001  2013-07  22.149300
## 271    A0001  2013-08   8.260577

# 各銘柄の1期先の予想配当データの読み込み
fy1D = pd.read_csv('./data/dividendData.csv')
print(fy1D)
##    ticker  fy1Dividend
## 0   A0001         52.0
## 1   A0002        148.0

       month   RMRF   SMB   HML    RF
0    1990-07  20.67 -1.56 -5.16  0.68
1    1990-08 -13.69 -3.63  0.98  0.66
2    1990-09   0.63  5.18  1.83  0.60
3    1990-10  -3.76 -5.30  1.84  0.68
4    1990-11 -19.52 -0.50  1.43  0.57
..       ...    ...   ...   ...   ...
295  2015-02   4.59 -2.00  0.73  0.00
296  2015-03   8.81 -4.46  0.11  0.00
297  2015-04   0.99 -0.16 -1.50  0.00
298  2015-05   4.60 -0.98 -0.49  0.00
299  2015-06   2.66  0.49 -0.74  0.00

[300 rows x 5 columns]
     ticker    month  close     return      share   RMRF    RF       RIRF
0     A0001  2008-07   2820  -7.571288   15815814   3.24  0.15  -7.721288
1     A0002  2008-07  12484  16.270840    5046567   3.24  0.15  16.120840
2     A0003  2008-07    738  -5.019305   22322538   3.24  0.15  -5.169305
3     A0004  2008-07    674  -7.544581   14930041   3.24  0.15  -7.694581
4     A0005  2008-07    712  -1.385042  408794784   3.24  0.15  -1.535042
...     ...      ...    ...        ...        ...    ...   ...        ...
46

### コード 4.2 ticker別にベータを推定する

In [7]:
# 単回帰でβを推定する関数
def calBeta(d):
    model = sm.OLS(d['RIRF'], sm.add_constant(d['RMRF']))  # 単回帰モデル
    res = model.fit()  # OLSによる回帰係数の推定
    return res.params['RMRF']  # 推定された定数項とRMRFの回帰係数


print(df[['ticker', 'RIRF', 'RMRF']])
##      ticker       RIRF   RMRF
## 0     A0001  -7.721288   3.24
## 1     A0002  16.120840   3.24

# 銘柄ごとにβを推定
beta = df[['ticker', 'RIRF', 'RMRF']].groupby('ticker').apply(calBeta)
beta.name = 'beta'
print(beta)
## ticker
## A0001    1.087651
## A0002    0.235348
##           ...
## A0079    0.449321
## A0080    0.826608
## Name: beta, Length: 77, dtype: float64

     ticker       RIRF   RMRF
0     A0001  -7.721288   3.24
1     A0002  16.120840   3.24
2     A0003  -5.169305   3.24
3     A0004  -7.694581   3.24
4     A0005  -1.535042   3.24
...     ...        ...    ...
4615  A0076   2.068965  11.22
4616  A0077   8.112515  11.22
4617  A0078   8.805032  11.22
4618  A0079  -3.631810  11.22
4619  A0080  11.488570  11.22

[4620 rows x 3 columns]
               0
ticker          
A0001   1.087651
A0002   0.235348
A0003   1.479954
A0004   1.073661
A0005   0.973627
...          ...
A0076   0.637097
A0077   0.351718
A0078   0.945899
A0079   0.449321
A0080   0.826608

[77 rows x 1 columns]


### コード 4.3 2013年6月の株価データから割安株を選択するプログラム

In [4]:
# ポートフォリを組成する銘柄の選択と投資ウェイトの計算
# 各銘柄の2013年6月末時点のV/Pを計算するため，2013-06を選択，予想配当やβも結合
df2 = df[df['month'] == '2013-06'].copy()
df2 = pd.merge(df2, fy1D, on='ticker')
df2 = pd.merge(df2, beta, on='ticker')

# 各銘柄のV/Pの推定値 (hatVP)の計算
# 年間の期待リターン（企業にとっての株式の資本コスト）単位は(%)
# マーケットリスクプレミアムは月間 0.3%とする
df2['expRet'] = (df2['RF'] + df2['beta'] * 0.3) * 12
# 毎期fy1Dividendの配当が永続すると仮定した理論株価
df2['hatV'] = df2['fy1Dividend'] / (df2['expRet'] / 100)
df2['hatVP'] = df2['hatV'] / df2['close']  # 2013年6月末時点で推定されたV/P

# hatVPが1を上回る割安株だけを抽出
df2 = df2[df2['hatVP'] > 1.0]
print('len(df2):', len(df2))  ## 43

# 抽出された割安株で構成されたポートフォリオを等加重と時価総額加重で運用するための投資ウェイトを計算しておく
df2['ew'] = 1.0 / len(df2)  # 等加重で運用する場合の各銘柄に対する投資ウェイト
df2['me'] = df2['close'] * df2['share']  # 各銘柄の時価総額 (= 株価 × 発行済み株式数)
df2['vw'] = df2['me'] / df2['me'].sum()  # 時価総額加重で運用する場合の各銘柄に対する投資ウェイト
print(df2)

len(df2): 43
   ticker    month   close     return      share   RMRF   RF       RIRF  \
0   A0001  2013-06    1219  21.052630   22784090  11.22  0.0  21.052630   
1   A0002  2013-06   13176   2.937500    5076411  11.22  0.0   2.937500   
2   A0003  2013-06    1738  17.591340   30687500  11.22  0.0  17.591340   
3   A0004  2013-06     384  34.265740   17246498  11.22  0.0  34.265740   
5   A0006  2013-06    5719  25.169620    4858141  11.22  0.0  25.169620   
7   A0008  2013-06    1245  10.079580    3485255  11.22  0.0  10.079580   
8   A0009  2013-06    1057   1.148325   28824464  11.22  0.0   1.148325   
10  A0011  2013-06     746  13.546420    5426584  11.22  0.0  13.546420   
12  A0013  2013-06     321  22.053230    9980456  11.22  0.0  22.053230   
13  A0014  2013-06     765  17.873650   47990692  11.22  0.0  17.873650   
14  A0015  2013-06     878   7.730062   27392816  11.22  0.0   7.730062   
16  A0017  2013-06    1023   8.483563    3495652  11.22  0.0   8.483563   
17  A0018  2

,ticker,month,close,return,share,RMRF,RF,RIRF,fy1Dividend,beta,expRet,hatV,hatVP,ew,me,vw
0,A0001,2013-06,1219,21.052630,22784090,11.22,0.0,21.052630,52.0,1.087651,3.915542,1328.040974,1.089451,0.023256,27773805710,0.002755
1,A0002,2013-06,13176,2.937500,5076411,11.22,0.0,2.937500,148.0,0.235348,0.847252,17468.230677,1.325761,0.023256,66886791336,0.006635
2,A0003,2013-06,1738,17.591340,30687500,11.22,0.0,17.591340,115.0,1.479954,5.327833,2158.476202,1.241931,0.023256,53334875000,0.005291
3,A0004,2013-06,384,34.265740,17246498,11.22,0.0,34.265740,20.0,1.073661,3.865179,517.440407,1.347501,0.023256,6622655232,0.000657
5,A0006,2013-06,5719,25.169620,4858141,11.22,0.0,25.169620,304.0,0.785453,2.827629,10751.055184,1.879884,0.023256,27783708379,0.002756
7,A0008,2013-06,1245,10.079580,3485255,11.22,0.0,10.079580,79.0,1.001285,3.604625,2191.628974,1.760345,0.023256,4339142475,0.000430
8,A0009,2013-06,1057,1.148325,28824464,11.22,0.0,1.148325,73.0,0.832449,2.996817,2435.917905,2.304558,0.023256,30467458448,0.003022
10,A0011,2013-06,746,13.546420,5426584,11.22,0.0,13.546420,33.0,0.760500,2.737801,1205.346792,1.615746,0.023256,4048231664,0.000402
12,A0013,2013-06,321,22.053230,9980456,11.22,0.0,22.053230,22.0,1.547776,5.571995,394.831671,1.230005,0.023256,3203726376,0.000318
13,A0014,2013-06,765,17.873650,47990692,11.22,0.0,17.873650,54.0,1.096532,3.947516,1367.948699,1.788168,0.023256,36712879380,0.003642


### コード4.4 ポートフォリオの評価

In [5]:
# 割安株で構成されたポートフォリオの運用結果
# 運用期間のデータに等加重と時価総額加重の場合のそれぞれの投資ウェイトを結合し，等加重平均リターンと加重平均リターンをそれぞれ求める
# tickerをキーに結合
futureRet2 = pd.merge(futureRet, df2[['ticker', 'ew', 'vw']], on='ticker')

futureRet2['ewRet'] = futureRet2['return'] * futureRet2['ew']
futureRet2['vwRet'] = futureRet2['return'] * futureRet2['vw']
print(futureRet2)
##     ticker    month     return        ew        vw     ewRet     vwRet
## 0    A0001  2013-07  22.149300  0.023256  0.002755  0.515100  0.061026
## 1    A0001  2013-08   8.260577  0.023256  0.002755  0.192106  0.022759

# 上で計算されたewRetとvwRetを月ごとに足し合わせれば完成
rsl = futureRet2[['month', 'ewRet', 'vwRet']].groupby('month').sum()
print(rsl)
##              ewRet      vwRet
## month
## 2013-07  -2.828633   4.815620
## 2013-08   6.110663  11.472223

    ticker    month     return        ew        vw     ewRet     vwRet
0    A0001  2013-07  22.149300  0.023256  0.002755  0.515100  0.061026
1    A0001  2013-08   8.260577  0.023256  0.002755  0.192106  0.022759
2    A0001  2013-09  15.508690  0.023256  0.002755  0.360667  0.042729
3    A0001  2013-10  -6.015038  0.023256  0.002755 -0.139885 -0.016573
4    A0001  2013-11 -13.085710  0.023256  0.002755 -0.304319 -0.036054
..     ...      ...        ...       ...       ...       ...       ...
511  A0078  2014-02  -6.577540  0.023256  0.000635 -0.152966 -0.004180
512  A0078  2014-03 -36.834570  0.023256  0.000635 -0.856618 -0.023406
513  A0078  2014-04  -3.488899  0.023256  0.000635 -0.081137 -0.002217
514  A0078  2014-05   3.708920  0.023256  0.000635  0.086254  0.002357
515  A0078  2014-06  73.291080  0.023256  0.000635  1.704444  0.046571

[516 rows x 7 columns]
             ewRet      vwRet
month                        
2013-07  -2.828633   4.815620
2013-08   6.110663  11.472223
2013